simple example of how to create a petstorm store, as discussed in chapter 7

In [1]:
SAMPLE_SIZE = 1

Define schema

In [2]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType

from petastorm.codecs import ScalarCodec, CompressedImageCodec, NdarrayCodec
from petastorm.etl.dataset_metadata import materialize_dataset
from petastorm.unischema import dict_to_spark_row, Unischema, UnischemaField

from pyspark.sql.functions import col
from pyspark.sql.types import *

from petastorm.spark import SparkDatasetConverter, make_spark_converter
from petastorm import TransformSpec 

import tensorflow as tf

/opt/conda/lib/python3.9/site-packages/petastorm/spark/spark_dataset_converter.py:28: FutureWarning: pyarrow.LocalFileSystem is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  from pyarrow import LocalFileSystem


In [3]:
# start Spark session:

spark = SparkSession \
    .builder \
    .appName("Create petastorm store") \
    .config("spark.memory.offHeap.enabled",True) \
    .config("spark.memory.offHeap.size","30g")\
    .getOrCreate()


spark.conf.set(SparkDatasetConverter.PARENT_CACHE_DIR_URL_CONF, 'file:/home/jovyan/petastorm_tmp_cache')

In [4]:
data_path = "images_data/silver/augmented"
# Read the data stored in parquet, limiting the dataset for the example
df_parquet = spark.read.parquet(data_path)
df = df_parquet.select(col("content"), col("label_index").cast(LongType())).limit(SAMPLE_SIZE)


In [5]:
df.printSchema()

root
 |-- content: binary (nullable = true)
 |-- label_index: long (nullable = true)



In [6]:
expected_df = df.collect()

In [12]:
expected_df[0][1]

3

In [8]:
import tensorflow as tf


converter = make_spark_converter(df)
with converter.make_tf_dataset(transform_spec=transform_spec_fn) as dataset:
        iterator = iter(dataset)
        print(iterator.element_spec)
        
        tensor = iterator.get_next()
        print(tensor)

/opt/conda/lib/python3.9/site-packages/petastorm/fs_utils.py:88: FutureWarning: pyarrow.localfs is deprecated as of 2.0.0, please use pyarrow.fs.LocalFileSystem instead.
  self._filesystem = pyarrow.localfs
Converting floating-point columns to float32


inferred_schema_view(content=TensorSpec(shape=(None,), dtype=tf.string, name=None), label_index=TensorSpec(shape=(None,), dtype=tf.int64, name=None))


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [9]:
# The schema defines how the dataset schema looks like
imageSchema = Unischema('caltech256schema', [
    UnischemaField('content', np.uint8, (224, 224, 3) ,False),
    UnischemaField('label_index', np.int32, (), ScalarCodec(LongType()), False)])

In [10]:
imageSchema

In [11]:
rows_rdd =  df.rdd.map(lambda x: dict_to_spark_row(imageSchema, x))

rows_rdd.take(1)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 8.0 failed 1 times, most recent failure: Lost task 0.0 in stage 8.0 (TID 22) (ed317257ff44 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/sql/types.py", line 1571, in __getattr__
    idx = self.__fields__.index(item)
ValueError: 'keys' is not in list

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.

In [ ]:
sc = spark.sparkContext
output_url = "images_data/petastorm"
rowgroup_size_mb = 256

    # Wrap dataset materialization portion. Will take care of setting up spark environment variables as
    # well as save petastorm specific metadata
rows_count = 10

with materialize_dataset(spark, output_url, imageSchema, rowgroup_size_mb):
        spark.createDataFrame(rows_rdd, imageSchema.as_spark_schema()) \
            .coalesce(10) \
            .write \
            .mode('overwrite') \
            .parquet(output_url)

## Barrier code example

In [ ]:
from pyspark import BarrierTaskContext


def stage_logic(row):
    context = BarrierTaskContext.get()
    # some logic that needs to be coordinated
    context.barrier()
    return row

In [ ]:
barrierRdd = df.rdd.barrier()
rdd = barrierRdd.mapPartitions(lambda x: stage_logic( x))

## to delete! 

In [ ]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.types import IntegerType

from petastorm.codecs import ScalarCodec, CompressedImageCodec, NdarrayCodec
from petastorm.etl.dataset_metadata import materialize_dataset
from petastorm.unischema import dict_to_spark_row, Unischema, UnischemaField

# The schema defines how the dataset schema looks like
HelloWorldSchema = Unischema('HelloWorldSchema', [
    UnischemaField('id', np.int32, (), ScalarCodec(IntegerType()), False),
    UnischemaField('image1', np.uint8, (128, 256, 3), CompressedImageCodec('png'), False),
    UnischemaField('array_4d', np.uint8, (None, 128, 30, None), NdarrayCodec(), False),
])


def row_generator(x):
    """Returns a single entry in the generated dataset. Return a bunch of random values as an example."""
    return {'id': x,
            'image1': np.random.randint(0, 255, dtype=np.uint8, size=(128, 256, 3)),
            'array_4d': np.random.randint(0, 255, dtype=np.uint8, size=(4, 128, 30, 3))}


def generate_petastorm_dataset(output_url='file://hello_world_dataset'):
    rowgroup_size_mb = 256

    spark = SparkSession.builder.config('spark.driver.memory', '2g').master('local[2]').getOrCreate()
    sc = spark.sparkContext

    # Wrap dataset materialization portion. Will take care of setting up spark environment variables as
    # well as save petastorm specific metadata
    rows_count = 10
    with materialize_dataset(spark, output_url, HelloWorldSchema, rowgroup_size_mb):

        rows_rdd = sc.parallelize(range(rows_count))\
            .map(row_generator)\
            .map(lambda x: dict_to_spark_row(HelloWorldSchema, x))

        spark.createDataFrame(rows_rdd, HelloWorldSchema.as_spark_schema()) \
            .coalesce(10) \
            .write \
            .mode('overwrite') \
            .parquet(output_url)
        
generate_petastorm_dataset()

In [ ]:
    # Cleanup
    converter_train.delete()
    converter_test.delete()
    spark.stop()